In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, svm, preprocessing, metrics
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 建Label dict
- key是FileID
- value是Label
- 方便最後塞Label用

In [58]:
df_train = pd.DataFrame(pd.read_csv('training-set.csv',header=None))
df_train.columns = ['FileID','Label']

dic = df_train.set_index('FileID').to_dict()

dic = dic['Label']

li = list(df_train['FileID'])

['0000e2398b12121a85166fed5fe2a3da',
 '0001fe8dce14ce099aa6ca8ea5026ea7',
 '00027f50019000accc492e5684efc818',
 '00028c9da3573ec50db74b44310ae507',
 '0003dc8130969abe688cadf5f14ea19f',
 '000462baadff8a02f7f6fef84d242eed',
 '0005e4c0263f52453438ddd0d55aef82',
 '0006ac9aadbdbf2fed5e62b4e463daa1',
 '0006d16543c1c4866dfa59fbbf7eedab',
 '0007819a84e047a78fbbd620557104ad',
 '0008720f6bfadf1aa4039a856eef3ecb',
 '000a00e6939f803bbf4aaf51b39048d0',
 '000b4096d7a0962245181f01e002b203',
 '000d282c41fc76964cdc31b805c464a7',
 '000d409d670038a868ed471a8334345d',
 '000f15bf6a8f0b51126ece0ce93af65c',
 '0010f818848455dfe997e69ee03518ef',
 '00117b31bd107c43123c8a0180dc036c',
 '0011d5f78c56a18dbb60a455c084ecca',
 '0012028fdd971d39caa88c8d0136334a',
 '001543427ff6300877e6a44e354588a6',
 '0015734d9e317de07623fd04ddaef05a',
 '00192fd48e421b51228e63845df46b9c',
 '001abb71230ddab7ff7016ccfac80d16',
 '001ac19de0eb93f585f74c2d207d9afc',
 '001fbbcf7a772e68e7ca1accf93e807d',
 '00204f1e3ee2102835b99b1cd372a8d5',
 

## 讀檔
1. 把之前merge好的大csv讀進來
2. Unnamed: 0欄位是沒用的，先刪掉
3. 增加一個count欄位，方便算比數（應該有更好的寫法）
4. 根據FileID group 幾個欄位出來
    - 'Cnum':每個FileID有對應到的不同的CustomerID
    - 'Pnum':每個FileID有對應到的不同的ProductID
    - 'df_guby_t_max':每個FileID出現的最後時間
    - 'df_guby_t_min':每個FileID第一次出現的時間
    - 'times':每個FileID出現的次數
    - 'time_diff'= df_guby_t_max - df_guby_t_min存在多長的時間，以天為單位
    - 'tzone1' :檔案在0~6點被開啟的比例
    - 'tzone2' :檔案在7~12點被開啟的比例
    - 'tzone3' :檔案在13~18點被開啟的比例
    - 'tzone4' :檔案在19~0點被開啟的比例
    - 'c_cut' 相關系列的，我把 CustomerID 出現的次數做了簡單的分群，第一群是出現10次以內，第二群是11~100，第三群101~1000，第四群1001~10000，第五群10001~100000，第六群大於100000次。'c_cutr' 相關系列是把 'c_cut'/Cnum變成比例，沒有取代'c_cut'而是新建欄位的用意是，我怕第五第六群萬一是重要的feature又被我稀釋掉的話就沒意義了。
    - 'c_cut1' :第一群Customer出現的次數
    - 'c_cut2' :第二群Customer出現的次數
    - 'c_cut3' :第三群Customer出現的次數
    - 'c_cut4' :第四群Customer出現的次數
    - 'c_cut5' :第五群Customer出現的次數
    - 'c_cut6' :第六群Customer出現的次數
    - 'c_cut1r' :第一群Customer出現的比例
    - 'c_cut2r' :第二群Customer出現的比例
    - 'c_cut3r' :第三群Customer出現的比例
    - 'c_cut4r' :第四群Customer出現的比例
    - 'c_cut5r' :第五群Customer出現的比例
    - 'c_cut6r' :第六群Customer出現的比例
    - 'freq' :tim_diff / times file 被點擊的頻率
5. 用上面建立好的dict把Label併進來
6. 把Label != 3的歸類為X(train data);Label == 3 的歸類為y(test data)
7. 用train_test_split把X分成訓練跟驗證資料

In [3]:
df_log = pd.DataFrame(pd.read_csv('total_log.csv'))

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_log = df_log.drop(['Unnamed: 0','Label'],axis = 1)

In [5]:
df_log['count'] = 1

df_log['apm'] = pd.to_numeric(df_log['Dtime'].str[11:-6])

In [6]:
df_cus = df_log.groupby('CustomerID', as_index = False)['count'].count()

In [16]:
df_cus2 = df_cus[['CustomerID','count']]

In [18]:
df_cus2 = pd.DataFrame(df_cus2)

In [54]:
cus_cut1 = list(df_cus2[df_cus2['count'] <= 10]['CustomerID'])
cus_cut2 = list(df_cus2[(df_cus2['count'] > 10) & (df_cus2['count'] <= 100)]['CustomerID'])
cus_cut3 = list(df_cus2[(df_cus2['count'] > 100) & (df_cus2['count'] <= 1000)]['CustomerID'])
cus_cut4 = list(df_cus2[(df_cus2['count'] > 1000) & (df_cus2['count'] <= 10000)]['CustomerID'])
#cus_cut5 = list(df_cus2[(df_cus2['count'] > 10000)]['CustomerID'])
cus_cut5 = list(df_cus2[(df_cus2['count'] > 10000) & (df_cus2['count'] <= 100000)]['CustomerID'])
cus_cut6 = list(df_cus2[(df_cus2['count'] > 100000)]['CustomerID'])

In [83]:
c1 = list(df_log[df_log['CustomerID'].isin(cus_cut1)].index.values)
c2 = list(df_log[df_log['CustomerID'].isin(cus_cut2)].index.values)
c3 = list(df_log[df_log['CustomerID'].isin(cus_cut3)].index.values)
c4 = list(df_log[df_log['CustomerID'].isin(cus_cut4)].index.values)
c5 = list(df_log[df_log['CustomerID'].isin(cus_cut5)].index.values)
c6 = list(df_log[df_log['CustomerID'].isin(cus_cut6)].index.values)

In [85]:
df_log['c_cut1'] = 0
df_log['c_cut2'] = 0
df_log['c_cut3'] = 0
df_log['c_cut4'] = 0 
df_log['c_cut5'] = 0 
df_log['c_cut6'] = 0 
df_log.loc[c1,'c_cut1'] = 1
df_log.loc[c2,'c_cut2'] = 1
df_log.loc[c3,'c_cut3'] = 1
df_log.loc[c4,'c_cut4'] = 1
df_log.loc[c5,'c_cut5'] = 1
df_log.loc[c6,'c_cut6'] = 1

In [87]:
df_cc = df_log.groupby(['FileID'])['c_cut1','c_cut2','c_cut3','c_cut4','c_cut5','c_cut6'].sum()

## TimeZone 處理

In [93]:
z1 = list(df_log[(df_log['apm']>0) & (df_log['apm']<=6)].index.values)
z2 = list(df_log[(df_log['apm']>6) & (df_log['apm']<=12)].index.values)
z3 = list(df_log[(df_log['apm']>12) & (df_log['apm']<=18)].index.values)
z4 = list(df_log[(df_log['apm']>18) | (df_log['apm']==0)].index.values)

In [95]:
df_log['tzone1'] = 0
df_log['tzone2'] = 0
df_log['tzone3'] = 0
df_log['tzone4'] = 0 
df_log.loc[z1,'tzone1'] = 1
df_log.loc[z2,'tzone2'] = 1
df_log.loc[z3,'tzone3'] = 1
df_log.loc[z4,'tzone4'] = 1

In [96]:
df_tz = df_log.groupby(['FileID'])['tzone1','tzone2','tzone3','tzone4'].sum()

## 最大、最小時間

In [97]:
df_guby_t_max = df_log.groupby('FileID')['QueryTS'].max()
df_guby_t_min = df_log.groupby('FileID')['QueryTS'].min()

In [11]:
# df_1 = df_log.groupby(['FileID','CustomerID','ProductID'])[['count']].sum()
# df_tmin = df_log.groupby(['FileID','CustomerID','ProductID'])[['QueryTS']].first()
# df_tmax = df_log.groupby(['FileID','CustomerID','ProductID'])[['QueryTS']].last()
# df_mid = df_log.groupby(['FileID','CustomerID','ProductID'])[['QueryTS']].median()

## CustomerID、ProductID unique 數量

In [98]:
df_guby = df_log.groupby('FileID').CustomerID.nunique()
df_gubyp = df_log.groupby('FileID').ProductID.nunique()

## File 出現的總次數

In [99]:
df_gubycoun = df_log.groupby('FileID')['count'].sum()

## 併 Feature 大表

In [100]:
total_data = pd.concat([df_guby, df_gubyp, df_guby_t_max, df_guby_t_min, df_gubycoun, df_tz, df_cc], axis=1)
total_data.columns = ['Cnum','Pnum','df_guby_t_max','df_guby_t_min','times','tzone1','tzone2','tzone3','tzone4','c_cut1','c_cut2','c_cut3','c_cut4','c_cut5','c_cut6']

## 算 File 的存在天數

In [102]:
total_data['time_diff'] = (total_data['df_guby_t_max'] - total_data['df_guby_t_min'])/(60*60*24)
#total_data = total_data.drop(['df_guby_t_max', 'df_guby_t_min'],axis = 1)

## 把 tzone 及 c_cut 轉成比例

In [103]:
total_data['tzone1'] = (total_data['tzone1'] / total_data['times'])*100
total_data['tzone2'] = (total_data['tzone2'] / total_data['times'])*100 
total_data['tzone3'] = (total_data['tzone3'] / total_data['times'])*100 
total_data['tzone4'] = (total_data['tzone4'] / total_data['times'])*100
total_data['c_cut1r'] = (total_data['c_cut1'] / total_data['Cnum'])*100
total_data['c_cut2r'] = (total_data['c_cut2'] / total_data['Cnum'])*100
total_data['c_cut3r'] = (total_data['c_cut3'] / total_data['Cnum'])*100
total_data['c_cut4r'] = (total_data['c_cut4'] / total_data['Cnum'])*100
total_data['c_cut5r'] = (total_data['c_cut5'] / total_data['Cnum'])*100
total_data['c_cut6r'] = (total_data['c_cut6'] / total_data['Cnum'])*100

## 算 freq 
- 這邊算頻率的時候我把時間單位換為分，怕天的話有些會太小而被算成0

In [105]:
total_data['freq'] = (total_data['time_diff'] * 24 * 60) / total_data['times']

## 存Feature大表

In [107]:
# 確定一下沒有null值
total_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81894 entries, 00008c73ee43c15b16c26b26398c1577 to ffff94db3a63e4fd7585ddc10a2ab044
Data columns (total 23 columns):
Cnum             81894 non-null int64
Pnum             81894 non-null int64
df_guby_t_max    81894 non-null int64
df_guby_t_min    81894 non-null int64
times            81894 non-null int64
tzone1           81894 non-null float64
tzone2           81894 non-null float64
tzone3           81894 non-null float64
tzone4           81894 non-null float64
c_cut1           81894 non-null int64
c_cut2           81894 non-null int64
c_cut3           81894 non-null int64
c_cut4           81894 non-null int64
c_cut5           81894 non-null int64
c_cut6           81894 non-null int64
time_diff        81894 non-null float64
c_cut1r          81894 non-null float64
c_cut2r          81894 non-null float64
c_cut3r          81894 non-null float64
c_cut4r          81894 non-null float64
c_cut5r          81894 non-null float64
c_cut6r          818

In [108]:
total_data.to_csv('feature_180307.csv', sep=',', encoding='utf-8')

## 分離 test data

In [ ]:
df_test = df_log[~df_log['FileID'].isin(li)]

In [ ]:
# total_data['Label'] = 3
# for i in range(len(total_data)):
#     try:
#         total_data['Label'][i] = dic[total_data.index[i]]
#     except:
#         total_data['Label'][i] = 3

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [72]:
X = train_data[train_data['Label'] != 3]
y = train_data[train_data['Label'] == 3]

In [76]:
X = X.drop('Label',axis = 1)

In [82]:
df_train = df_train.set_index('FileID')

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, df_train, random_state = 7, test_size = 0.25)

In [ ]:
# x = train_data.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df = pd.DataFrame(x_scaled)

## GBM
- 裝XGB的時候碰到了問題，所以先用GBM做了
- 做出來的預測是0,1不是機率（有點大的問題）
- train的時候沒有做很多的校調，一次就傳上去了（結果只有0.5哈哈哈哈（別打我

In [94]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [90]:
clf = GradientBoostingClassifier(n_estimators=500,max_depth=10,min_samples_split=5,min_samples_leaf=3)

In [91]:
clf.fit(X_train,y_train)

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=3, min_samples_split=5,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [92]:
pred_y = clf.predict(X_test)

In [95]:
accuracy = accuracy_score(y_test, pred_y )
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.99%


In [101]:
y_pred = clf.predict(y)

In [100]:
y = y.drop('Label',axis = 1)

In [105]:
y['Probability'] = y_pred

In [112]:
y_final = y.drop(['Cnum','Pnum','times','time_diff'],axis = 1)

In [115]:
y_final['FileID'] = y_final.index

In [119]:
df_y = y_final.reset_index(drop=True)

In [121]:
df_y = df_y[['FileID', 'Probability']]

In [123]:
df_y.to_csv('tbrain_v2.csv', sep=',', encoding='utf-8')